In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, AutoModelForSequenceClassification, pipeline, trainer
from transformers import Trainer, TrainingArguments



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelname = "liam168/c2-roberta-base-finetuned-dianping-chinese"
tokenizer = AutoTokenizer.from_pretrained(modelname)

max_len = 300


In [4]:


model = AutoModelForSequenceClassification.from_pretrained(modelname)

# raw_datasets = load_dataset("imdb")

In [5]:
def tokenize_function(examples):
    return tokenizer(examples['text'], max_length=max_len, padding='max_length', return_tensors = 'pt')

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
chn_train = load_dataset('seamew/ChnSentiCorp')
# BATCH=32
# MAX_SEQLEN=300
# LR=5e-5
# EPOCH=10

Using custom data configuration default
Reusing dataset chn_senti_corp (/home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:

def process_data(data):
    # max length is about 306..., truncation and padding are needed
    return tokenizer(data["text"], max_length=max_len, padding='max_length', truncation=True)


In [9]:
tokenized_datasets = chn_train.map(process_data)

  0%|          | 0/9600 [00:00<?, ?ex/s]

  0%|          | 0/1200 [00:00<?, ?ex/s]

  0%|          | 0/1200 [00:00<?, ?ex/s]

In [10]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=2)


In [12]:

num_epoch = 3
per_device_batch = 48

training_args = TrainingArguments(
  'finetune_trainer', 
  evaluation_strategy='epoch',
  num_train_epochs=num_epoch, 
  per_device_train_batch_size=per_device_batch, 
  per_device_eval_batch_size=per_device_batch
)

In [13]:
model.to(device)
trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [14]:
# print(fine_tune_train.columns)
torch.cuda.empty_cache()
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 9600
  Num Epochs = 3
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 300
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.182103
2,No log,0.175967
3,No log,0.169414


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=0.14193365732828775, metrics={'train_runtime': 291.7502, 'train_samples_per_second': 98.715, 'train_steps_per_second': 1.028, 'total_flos': 4439999059200000.0, 'train_loss': 0.14193365732828775, 'epoch': 3.0})

In [16]:
import os
os.makedirs("./dianping-finetuned-1")

FileExistsError: [Errno 17] File exists: './dianping-finetuned-1'

In [17]:
model.save_pretrained("./dianping-finetuned-1")

Configuration saved in ./dianping-finetuned-1/config.json
Model weights saved in ./dianping-finetuned-1/pytorch_model.bin


In [18]:
model = AutoModelForSequenceClassification.from_pretrained("./dianping-finetuned-1")

loading configuration file ./dianping-finetuned-1/config.json
Model config BertConfig {
  "_name_or_path": "./dianping-finetuned-1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading weights file ./dianping-finetuned-1/pytorch_model.bin

In [19]:
danmuku = pd.read_csv('test_dataframe.csv', nrows=200)
danmuku.shape

(200, 3)

In [20]:
danmuku['label'] = danmuku['label'].apply(lambda x: 1 if x < 0 else x)
for i in danmuku['label']:
    if i < 0:
        print(-1)

In [21]:
danmuku = danmuku.loc[:, ~danmuku.columns.str.contains('^Unnamed')]


In [22]:
from datasets import Dataset
danmuku_dataset = Dataset.from_pandas(danmuku)


In [23]:
tokenized_danmuku_dataset = danmuku_dataset.map(process_data)


  0%|          | 0/200 [00:00<?, ?ex/s]

In [24]:
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [25]:
testor = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_danmuku_dataset.shuffle(seed=42).select(range(160)),
    eval_dataset=tokenized_danmuku_dataset.shuffle(seed=42).select(range(160, 200)),
    compute_metrics=compute_metrics,
)

In [26]:
torch.cuda.empty_cache()

In [27]:
model.to(device)
testor.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 160
  Num Epochs = 3
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.805682,0.725000
2,No log,0.607274,0.750000
3,No log,0.569298,0.725000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

TrainOutput(global_step=6, training_loss=1.1337711811065674, metrics={'train_runtime': 5.1378, 'train_samples_per_second': 93.425, 'train_steps_per_second': 1.168, 'total_flos': 73999984320000.0, 'train_loss': 1.1337711811065674, 'epoch': 3.0})

In [28]:
testor.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.5692975521087646,
 'eval_accuracy': 0.725,
 'eval_runtime': 0.2151,
 'eval_samples_per_second': 185.966,
 'eval_steps_per_second': 4.649,
 'epoch': 3.0}

In [73]:
small_train_dataset[0]['attention_mask']


[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
